In [21]:
import pandas as pd
from pathlib import Path
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.porter import PorterStemmer
import re
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import xgboost as xgb
import dalex as dx
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
df = pd.read_json(Path.cwd() / "final_df" / "final_df.json")

In [3]:
df

,title,link,citations,text
0,Continual Reinforcement Learning with TELLA,http://arxiv.org/pdf/2208.04287v1,2,Workshop Track - 1st Conference on Lifelong Le...
1,An exact mapping between the Variational Renor...,http://arxiv.org/pdf/1410.3831v1,295,arXiv:1410.3831v1 [stat.ML] 14 Oct 2014An ex...
2,Learning Generative Models across Incomparable...,http://arxiv.org/pdf/1905.05461v2,69,Learning Generative Models across Incomparable...
3,On the Generalization Ability of Online Learni...,http://arxiv.org/pdf/1305.2505v1,74,On the Generalization Ability of Online Learni...
4,Geometric Understanding of Deep Learning,http://arxiv.org/pdf/1805.10451v2,110,Geometric Understanding of Deep Learning\nNa L...
...,...,...,...,...
1230,Musical Word Embedding: Bridging the Gap betwe...,http://arxiv.org/pdf/2008.01190v1,4,Musical Word Embedding: Bridging the Gap betwe...
1231,Metrics for Multi-Class Classification: an Ove...,http://arxiv.org/pdf/2008.05756v1,409,METRICS FOR MULTI -CLASS CLASSIFICATION :ANOVE...
1232,Intelligence plays dice: Stochasticity is esse...,http://arxiv.org/pdf/2008.07496v1,5,Intelligence plays dice: Stochasticity is esse...
1233,Can AutoML outperform humans? An evaluation on...,http://arxiv.org/pdf/2009.01564v2,24,Can AutoML outperform humans? An evaluation \n...


In [4]:
df['text'].isna().sum()

12

In [5]:
df.dropna(subset=['text'], inplace=True)

In [74]:
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(text: str):
    return ' '.join([stemmer.stem(word) for word in tokenizer.tokenize(' '.join([re.sub(r'[\s\d]+', ' ', word) for word in text.split()]))])


In [75]:
df['text_pp'] = df['text'].apply(lambda x: preprocess_text(x))

In [76]:
tf_vectorizer = CountVectorizer(ngram_range = (1, 3),
                                max_df = 0.75,
                                min_df = 0.01,
                                tokenizer = tokenizer.tokenize,
                                stop_words='english'
)

In [77]:
df.head()

,title,link,citations,text,text_pp
0,Continual Reinforcement Learning with TELLA,http://arxiv.org/pdf/2208.04287v1,2,Workshop Track - 1st Conference on Lifelong Le...,workshop track st confer on lifelong learn age...
1,An exact mapping between the Variational Renor...,http://arxiv.org/pdf/1410.3831v1,295,arXiv:1410.3831v1 [stat.ML] 14 Oct 2014An ex...,arxiv v stat ml oct an exact map between the v...
2,Learning Generative Models across Incomparable...,http://arxiv.org/pdf/1905.05461v2,69,Learning Generative Models across Incomparable...,learn gener model across incompar space charlo...
3,On the Generalization Ability of Online Learni...,http://arxiv.org/pdf/1305.2505v1,74,On the Generalization Ability of Online Learni...,on the gener abil of onlin learn algorithm for...
4,Geometric Understanding of Deep Learning,http://arxiv.org/pdf/1805.10451v2,110,Geometric Understanding of Deep Learning\nNa L...,geometr understand of deep learn na lei zhongx...


In [78]:
tf = tf_vectorizer.fit_transform(df['text_pp'])

In [79]:
#params = {
#    'n_components': list(range(5, 150, 5)),
#    'learning_decay': np.arange(0.6, 0.9, 0.1),
#    'learning_offset': np.arange(10, 150, 10),
#    'perp_tol': [0.1],
#    'max_iter': [20],
#    'mean_change_tol': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
#    'max_doc_update_iter': [100]
#}
best_params = {'perp_tol': 0.1,
 'n_components': 5,
 'mean_change_tol': 0.01,
 'max_iter': 200,
 'max_doc_update_iter': 100,
 'learning_offset': 30,
 'learning_decay': 0.6,
 'verbose': 2,
 'n_jobs': -1}

lda = LatentDirichletAllocation(**best_params)
#rsearch = RandomizedSearchCV(lda, params, n_iter=30, verbose=3, cv=3, error_score='raise')
#rsearch.fit(tf)
lda.fit(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   13.6s remaining:   13.6s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   13.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 200


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.6s remaining:    1.6s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 200


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.0s remaining:    2.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


...

iteration: 199 of max_iter: 200


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished


iteration: 200 of max_iter: 200


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s finished


LatentDirichletAllocation(learning_decay=0.6, learning_offset=30, max_iter=200,
                          mean_change_tol=0.01, n_components=5, n_jobs=-1,
                          verbose=2)

In [11]:
#best = rsearch.best_estimator_

In [80]:
#best.perplexity(tf)
lda.perplexity(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s finished


10897.208046380869

In [82]:
pickle.dump(lda, open('model.pkl', 'wb'))
#lda = pickle.load(open('model.pkl', 'rb'))

In [81]:
for index, component in enumerate(lda.components_): #taking model's components, i.e. P(w|z)
    zipped = zip(tf_vectorizer.get_feature_names(), component) #taking together tokens' names with components
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:10] #top 10 terms per topic
    top_terms_list=list(dict(top_terms_key).keys()) #taking only tokens, no weights

    print("Topic "+str(index)+": ",top_terms_list) #prints top 10 tokens per topic

E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Topic 0:  ['uni', 'uni uni', 'attack', 'adversari', 'privaci', 'commun', 'feder', 'secur', 'g g', 'g g g']
Topic 1:  ['layer', 'quantum', 'dataset', 'error', 'linear', 'gradient', 'nn', 'rate', 'kernel', 'di']
Topic 2:  ['dataset', 'ml', 'http', 'decis', 'pp', 'human', 'doi', 'class', 'tree', 'user']
Topic 3:  ['d d', 'u', 'z', 'q', 'bound', 'ct', 'θ', 'theorem', 'xi', 'x x']
Topic 4:  ['represent', 'polici', 'imag', 'meta', 'agent', 'reinforc', 'graph', 'domain', 'reward', 'reinforc learn']


In [83]:
lda_output = lda.transform(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.0s remaining:    1.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.1s finished


In [84]:
topics = [f'Topic {i}' for i in range(lda.n_components)]
docnames = df.index.tolist()
topics_df = pd.DataFrame(lda_output, columns=topics, index=docnames)

In [85]:
topics_df['citations'] = df['citations']

In [86]:
xg_model = xgb.XGBRegressor()
kf = KFold(n_splits=7, random_state=420602, shuffle=True)
cross_val_score(xg_model, topics_df[[f'Topic {i}' for i in range(lda.n_components)]], topics_df['citations'], scoring='neg_mean_absolute_percentage_error')

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with th

array([-14.17893892, -16.80485857, -12.56728666, -21.34428859,
       -22.20727879])

In [87]:
xg_model.fit(topics_df[[f'Topic {i}' for i in range(lda.n_components)]], topics_df['citations'])

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [88]:
exp = dx.Explainer(xg_model, topics_df[['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4']], topics_df['citations'])

Preparation of a new explainer is initiated

  -> data              : 1223 rows 5 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1223 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x000001727C9D9AF0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = -20.5, mean = 84.5, max = 1.17e+04
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -46.0, mean = 0.116, max = 98.0
  -> model_info        : package xgboost

A new explainer has been created!


In [89]:
agg_profiles = exp.model_profile().plot(show=False)
agg_profiles.write_html('agg_profiles.html')

Calculating ceteris paribus: 100%|██████████| 5/5 [00:00<00:00, 34.97it/s]


In [90]:
agg_profiles

In [91]:
var_importance = exp.model_parts().plot(show=False)
var_importance.write_html('var_importance.html')

In [92]:
var_importance

In [93]:
for index, component in enumerate(lda.components_): #taking model's components, i.e. P(w|z)
    zipped = zip(tf_vectorizer.get_feature_names(), component) #taking together tokens' names with components
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:10] #top 10 terms per topic
    top_terms_list=list(dict(top_terms_key).keys()) #taking only tokens, no weights

    print("Topic "+str(index)+": ",top_terms_list) #prints top 10 tokens per topic

E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Topic 0:  ['uni', 'uni uni', 'attack', 'adversari', 'privaci', 'commun', 'feder', 'secur', 'g g', 'g g g']
Topic 1:  ['layer', 'quantum', 'dataset', 'error', 'linear', 'gradient', 'nn', 'rate', 'kernel', 'di']
Topic 2:  ['dataset', 'ml', 'http', 'decis', 'pp', 'human', 'doi', 'class', 'tree', 'user']
Topic 3:  ['d d', 'u', 'z', 'q', 'bound', 'ct', 'θ', 'theorem', 'xi', 'x x']
Topic 4:  ['represent', 'polici', 'imag', 'meta', 'agent', 'reinforc', 'graph', 'domain', 'reward', 'reinforc learn']


In [94]:
exp.model_performance()

,mse,rmse,r2,mae,mad
XGBRegressor,253.769159,15.930134,0.998542,10.415174,6.718426
